In [1]:
import pickle
import cv2
import mediapipe as mp
import numpy as np

2024-12-05 10:30:36.369932: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733387436.413986   19858 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733387436.429856   19858 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-05 10:30:36.489511: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
model_dict = pickle.load(open('./model.pickle', 'rb'))
model = model_dict['model']

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

labels_dict = {0: 'A', 1: 'B', 2: 'L'}

I0000 00:00:1733387443.793671   19858 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1733387443.797875   19998 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1), renderer: Mesa Intel(R) UHD Graphics 620 (KBL GT2)


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1733387443.872657   19981 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733387443.925620   19986 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [7]:
cap = cv2.VideoCapture(0)

while True:
    X, Y = [], []
    data = []

    ret, frame = cap.read()
    assert ret

    H, W, _ = frame.shape

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    detected_hands = hands.process(frame_rgb)

    if detected_hands.multi_hand_landmarks:
        for hand_landmarks in detected_hands.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())
        
        for hand_landmarks in detected_hands.multi_hand_landmarks:
            for i in range(len(hand_landmarks.landmark)):
                # x & y between 0 and 1
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y

                X.append(x)
                Y.append(y)
        
            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y

                data.append(x-min(X))
                data.append(y-min(Y))
                '''
                Why This Step is Useful? Even though the coordinates are between 0 and 1, this normalization process:

                Helps your model focus on the shape and gesture rather than the absolute position of the hand.
                Improves robustness in real-time applications where the hand moves around the frame.
                Without this step, two identical gestures at different locations would have different coordinate values, which could confuse the model during training.
                '''
        
        # draw rectangle around hand
        x1 = int(min(X)*W)-10
        y1 = int(min(Y)*H)-10
        x2 = int(max(X)*W)-10
        y2 = int(max(Y)*H)-10

        pred = model.predict([np.asarray(data)])
        class_name = labels_dict[int(pred[0])]

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
        cv2.putText(frame, class_name, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)

    cv2.imshow('frame', frame)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()